In [1]:
# Cell 1: Kiểm tra môi trường
import sys
print("Python executable:", sys.executable)
print("Python version:", sys.version)

import torch
print("✅ PyTorch version:", torch.__version__)
print("✅ CUDA available:", torch.cuda.is_available())

Python executable: /root/miniconda3/envs/chest-xray/bin/python
Python version: 3.11.14 (main, Oct 21 2025, 18:31:21) [GCC 11.2.0]
✅ PyTorch version: 2.8.0+cu129
✅ CUDA available: True


In [2]:
# Cell 2: Import model
import os
import sys

# Thêm đường dẫn project root
project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from src.models.cnn import SimpleCNN

print("✅ Import model thành công!")

✅ Import model thành công!


In [4]:
# Cell 3: Test model
model = SimpleCNN(num_classes=4)
print("=" * 60)
print("MODEL ARCHITECTURE")
print("=" * 60)
print(model)

MODEL ARCHITECTURE
SimpleCNN(
  (conv1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv4): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 

In [5]:
# Cell 4: Test với dummy input
dummy_input = torch.randn(2, 3, 224, 224)
print(f"Input shape: {dummy_input.shape}")

output = model(dummy_input)
print(f"Output shape: {output.shape}")
print(f"Output:\n{output}")

Input shape: torch.Size([2, 3, 224, 224])
Output shape: torch.Size([2, 4])
Output:
tensor([[-0.1573,  0.0553, -0.5254,  0.2935],
        [ 0.0824, -0.2279,  0.2370,  0.3486]], grad_fn=<AddmmBackward0>)


In [6]:
# Cell 5: Đếm parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")
print(f"Model size: ~{total_params * 4 / (1024**2):.2f} MB")

Total parameters: 26,082,052
Trainable parameters: 26,082,052
Model size: ~99.50 MB
